In [1]:
'''
Modified from https://github.com/pytorch/vision.git
'''
from __future__ import print_function, division
import math

import torch.nn as nn
import torch.nn.init as init
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import models
from torch.autograd import Variable
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime

plt.ion()   # interactive mode

In [2]:

__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]


class VGG(nn.Module):
    '''
    VGG model 
    '''
    def __init__(self, features):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10),
        )
         # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 
          512, 512, 512, 512, 'M'],
}

In [3]:

def vgg11():
    """VGG 11-layer model (configuration "A")"""
    return VGG(make_layers(cfg['A']))


def vgg11_bn():
    """VGG 11-layer model (configuration "A") with batch normalization"""
    return VGG(make_layers(cfg['A'], batch_norm=True))


def vgg13():
    """VGG 13-layer model (configuration "B")"""
    return VGG(make_layers(cfg['B']))


def vgg13_bn():
    """VGG 13-layer model (configuration "B") with batch normalization"""
    return VGG(make_layers(cfg['B'], batch_norm=True))


def vgg16():
    """VGG 16-layer model (configuration "D")"""
    return VGG(make_layers(cfg['D']))


def vgg16_bn():
    """VGG 16-layer model (configuration "D") with batch normalization"""
    return VGG(make_layers(cfg['D'], batch_norm=True))


def vgg19():
    """VGG 19-layer model (configuration "E")"""
    return VGG(make_layers(cfg['E']))


def vgg19_bn():
    """VGG 19-layer model (configuration 'E') with batch normalization"""
    return VGG(make_layers(cfg['E'], batch_norm=True))

In [4]:
model = vgg11()

In [5]:
model.features = torch.nn.DataParallel(model.features)

In [6]:
model.cuda()

VGG(
  (features): DataParallel(
    (module): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): ReLU(inplace=True)
      (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (14): ReLU(inplace=True)
      (15): MaxPool2d(kernel

## cifar10 Dataset

In [4]:
batch_size = 128
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=batch_size, shuffle=True)

val_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=batch_size, shuffle=False)

# define loss function (criterion) and pptimizer
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified


In [5]:
import time
import os
import copy
import csv
import pandas as pd
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d')

In [6]:
training_round = 10
model_name = 'resnet18'

In [7]:
dataloaders_dict = {'train': train_loader, 'val':val_loader}
dataset_sizes = {'train': len(train_loader.dataset.data), 'val': len(val_loader.dataset.data) }
class_names = train_loader.dataset.classes

In [8]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25, is_inception=False):
    since = time.time()
    if not os.path.exists('./data/26-10-2021/Train1/pytorch/{}'.format(model_name)):
        os.makedirs('./data/26-10-2021/Train1/pytorch/{}'.format(model_name))
    data_file = open('./data/26-10-2021/Train1/pytorch/{}/torch_exp_{}_{}_{}.csv'.format(model_name, model_name, date, training_round), mode='w+', newline='', encoding='utf-8')
    data_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data_writer.writerow(['Epoch','Train_loss', 'Train_acc', "val_loss", "Val_acc", 'time','Elapse_time','date'])

    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        since_1 = time.time()

        # Each epoch has a training and validation phase
        #data_writer.writerow(['Model','type', 'Dataset', 'Train_loss', 'Train_acc', "val_loss", "Val_acc"])
        rows = [epoch]
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                inputs = inputs.float()
                #labels = labels.float()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                   # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        #print(criterion)
                        loss = criterion(outputs, labels)
                    #outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    #loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            rows.append(phase)
            rows.append('Loss: {:.4f}'.format(epoch_loss))
            rows.append('Acc: {:.4f}'.format(epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
        time_elapsed_1 = time.time() - since_1
        print()
        rows.append(time.time())
        rows.append('{:.0f}m {:.0f}s'.format(time_elapsed_1 // 60, time_elapsed_1 % 60))
        data_writer.writerow(rows)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    data_writer.writerow(['', "", 'Best val Acc: {:4f}'.format(best_acc), time.time(),'Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60),''])

    data_file.close()
    return model, val_acc_history

In [12]:
#criterion = criterion.cuda()
#optimizer = torch.optim.SGD(model.parameters(), 0.001,
#                                momentum=0.9)

In [13]:
#num_epochs = 50


In [14]:
# Setup the loss fxn
#from torch.optim import lr_scheduler
# criterion = nn.CrossEntropyLoss()
# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
# Train and evaluate
#model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer, exp_lr_scheduler, num_epochs=num_epochs, is_inception=(model_name=="inception"))

In [9]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [10]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
#data_dir = "./data/hymenoptera_data"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 10

# Batch size for training (change depending on how much memory you have)
batch_size = 128

# Number of epochs to train for
num_epochs = 50

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [12]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

In [14]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/49
----------


/usagers3/opmos/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 1.3093 Acc: 0.5395
val Loss: 0.9496 Acc: 0.6652

Epoch 1/49
----------
train Loss: 0.8987 Acc: 0.6847
val Loss: 0.8076 Acc: 0.7165

Epoch 2/49
----------
train Loss: 0.7716 Acc: 0.7304
val Loss: 0.7063 Acc: 0.7524

Epoch 3/49
----------
train Loss: 0.6942 Acc: 0.7571
val Loss: 0.6553 Acc: 0.7703

Epoch 4/49
----------
train Loss: 0.6397 Acc: 0.7758
val Loss: 0.6371 Acc: 0.7807

Epoch 5/49
----------
train Loss: 0.5999 Acc: 0.7894
val Loss: 0.5986 Acc: 0.7923

Epoch 6/49
----------
train Loss: 0.5666 Acc: 0.8006
val Loss: 0.5852 Acc: 0.7995

Epoch 7/49
----------
train Loss: 0.5197 Acc: 0.8164
val Loss: 0.5648 Acc: 0.8061

Epoch 8/49
----------
train Loss: 0.5069 Acc: 0.8226
val Loss: 0.5612 Acc: 0.8087

Epoch 9/49
----------
train Loss: 0.5045 Acc: 0.8215
val Loss: 0.5592 Acc: 0.8086

Epoch 10/49
----------
train Loss: 0.5017 Acc: 0.8248
val Loss: 0.5581 Acc: 0.8108

Epoch 11/49
----------
train Loss: 0.4949 Acc: 0.8251
val Loss: 0.5557 Acc: 0.8093

Epoch 12/49
----------
t

In [15]:
torch.save(model_ft, './data/26-10-2021/Train1/pytorch/{}/torch_exp_{}_{}_{}.pth'.format(model_name, model_name, date, training_round))

### 

In [29]:
path = './data/23-08-2021/'
# Model class must be defined somewhere
since_0 = time.time()
#model_name = 'torch_resnet50-cifar10_2021-08-19-15:05:17'
#model_name = 'torch_resnet18_cifar10_2021-08-20-15:17:40'
model_name = 'torch_vgg_cifar10_2021-09-30-17:51:30'
#model_name = 'torch_alexnet_cifar10_2021-08-20-15:26:26'
#model_name = 'torch_alexnet_cifar10_2021-08-20-15:26:26'
#model_name = 'torch_Lenet5-mnist_2021-09-03-13:54:37'
model = torch.load(path+model_name+'.pth')
#resnet50_model.eval()
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+model_name+'.pth')
size0

515298923

In [30]:
size1 = os.path.getsize(path+model_name+'.pth')
size1

515298923

In [31]:
import onnx
import onnxruntime
import time

In [32]:
import caffe2.python.onnx.backend as onnx_caffe2_backend

In [38]:
def to_onnx(i, x, abs_errors,rel_errors, t0_list, t1_list, t2_list, t3_list, s_list):
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
   
    print("converting for batch: ", i)
    model.to(device)
    
    torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    since_1 = time.time()
    #model = torch.load(path+model_name+'.pth')
    torch_out = model(x)
    t_elapsed_1 = time.time() - since_1
    # Export the model
    since_1 = time.time()
    torch.onnx._export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "./data/ONNX/torch/model_ft-{}.onnx".format(model_name),   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  #operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})
    t_elapsed_2 = time.time() - since_1
    
    
    onnx_model = onnx.load("./data/ONNX/torch/model_ft-{}.onnx".format(model_name))
    onnx.checker.check_model(onnx_model)
    
    size2 = os.path.getsize("./data/ONNX/torch/model_ft-{}.onnx".format(model_name))
    s_list.append(size2)
    
    def to_numpy(tensor):
        return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    #try:
    ort_session = onnxruntime.InferenceSession("./data/ONNX/torch/model_ft-{}.onnx".format(model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
    ort_outs = ort_session.run(None, ort_inputs)
    #except:
    #    since_1 = time.time()
    #    prepared_backend = onnx_caffe2_backend.prepare(onnx_model)
    #    W = {model.graph.input[0].name: to_numpy(x)}
    #    ort_outs = prepared_backend.run(W)
        
   
    t_elapsed_3 = time.time() - since_1
    # compare ONNX Runtime and PyTorch results
    print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    t0_list.append(t_elapsed_0)
    t1_list.append(t_elapsed_1)
    t2_list.append(t_elapsed_2)
    t3_list.append(t_elapsed_3)
    
    abs_err = np.absolute(to_numpy(torch_out)-ort_outs[0])
    rel_err = np.absolute(to_numpy(torch_out)-ort_outs[0])/ np.absolute(ort_outs[0])
    abs_errors.append(abs_err)
    rel_errors.append(rel_err)
    
    return (abs_err, rel_err)

In [39]:
def _lets_convert(data):
    device = torch.device("cpu")
    since = time.time()
    list_converted = []
    t0_list = []
    t1_list = []
    t2_list = []
    t3_list = []
    s_list = []
    abs_errors = []
    rel_errors = []
    for i, (images, labels) in enumerate(data):
        #torch.cuda.empty_cache()
        images = images.to(device)
        list_converted.append(to_onnx(i, images, abs_errors,rel_errors, t0_list, t1_list, t2_list, t3_list, s_list))
        if i == 15:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s,  Loading Pytorch: {}, Pytorch time: {:.4f}, conversion time: {:.4f}, onnx runtime: {:.4f}'.format(
        time_elapsed // 60, time_elapsed % 60, np.mean(np.array(t0_list)), np.mean(np.array(t1_list)), np.mean(np.array(t2_list)), np.mean(np.array(t3_list))) )
    
    #return list_converted
    return list_converted, abs_errors, rel_errors, 'Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60), np.mean(np.array(t0_list)), np.mean(np.array(t1_list)), np.mean(np.array(t2_list)), np.mean(np.array(t3_list)), np.mean(np.array(s_list))

In [40]:
#list_converted = _lets_convert(valloader)
#list_converted = _lets_convert(test_ds)
model_name = 'VGG-torch'
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
import pandas as pd 
for batch_size in [1, 5,10,20,30,40,50,60,70,80,90,100,128, 150, 200, 250,300,350, 400, 450, 500]:
    print("################ Batch size: ", batch_size)
    val_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=batch_size, shuffle=False)
    # specify the image classes
    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

    list_converted, abs_errors, rel_errors, total_time, t0, t1, t2, t3, file_size = _lets_convert(val_loader)
    for i in range(len(abs_errors)):
        if i == 0:
            abs_array = abs_errors[i]
            rel_array = rel_errors[i]
        else:
            np.append(abs_array, abs_errors[i])
            np.append(rel_array, rel_errors[i])
            
    abs_list = []
    rel_list = []
    model_list = []
    batch_list = []
    summary_list = ['Modelsize:{}, Conversion: {}, Loading: {}, t1: {}, conversion time: {}, onnx runtime: {}, onnx filesize: {}'.format(size0, total_time, t0, t1, t2, t3, file_size)]
    for i in range(len(abs_array)):
        abs_list.append(abs_array[i][0])
        rel = rel_array[i][0]
        if rel == np.inf or rel == -np.inf:
            rel = 0.0
        rel_list.append(rel)
        batch_list.append(batch_size)
        model_list.append(model_name)
        if i >= len(summary_list):
            summary_list.append('')
    print(len(summary_list), len(rel_list))
    data = pd.DataFrame({'model':model_list,'batch_size': batch_list, 'abs_errors':abs_list, 'rel_errors':rel_list, 'summary': summary_list})
    data.to_csv('./data/errors/torch2/tf_errors_{}_{}.csv'.format(model_name, batch_size))

################ Batch size:  1
converting for batch:  0


RuntimeError: Unsupported: ONNX export of operator adaptive_avg_pool2d, since output size is not factor of input size. Please feel free to request support or submit a pull request on PyTorch GitHub.